
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #2


Dans le 2nd jeu d'essai reçu de 956 actions, l'explosion combinatoire rend impossible le traitement car il existerait 6.090821257124996 e+287 combinaisons.

Il faut donc procéder autrement.
L'algorithme du sac à dos ou l'on parcourt un arbre des cas possibles en décidant de prendre ou non une action peut être optimisé.



### 1. Préparation des données



In [3]:
import csv as csv
import re as re
import time



In [4]:
try:
    import big_o
except ModuleNotFoundError:
    print('cherche',ModuleNotFoundError)
    !pip install --upgrade pip
    !pip install big_o
    import big_o

In [5]:
# constant
FILE = "data/p7-20-shares.csv" 
FIELDNAMES = ['name', 'cost', 'profit_share', 'profit']


BUDGET = 50000

In [6]:
 def fn_timer(function):
        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        print ("Total time running %s: %s nanoseconds" %
               (function.__name__, str(t1-t0))
               )
        return result
    return function_timer

In [7]:
# nettoyer les données des fichiers en entrée
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

### Amélioration des données

Sur python le travail avec float est plus couteux qu'en entier.
Une solution est alors de multiplier par 100 cout, budget et profit car cela ne change pas
le résultat.


In [8]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=';', doublequote=False)
        # skip the header
        next(csv_reader)
        for idx, line in enumerate(csv_reader):
            clean_data = True
            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False
            if line[FIELDNAMES[1]] != "":
                cout = 100 * int(clean_char(line[FIELDNAMES[1]]))
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
            if line[FIELDNAMES[3]] != "":
                gain = int(100 * float(clean_char(line[FIELDNAMES[3]])))
            else:
                print(f" line {idx} had missing profit data; dropped.")
                clean_data = False
            if (gain < 0) or (cout < 0):
                print(f" line {idx} had negative value; accepted but pls check.")
            if (clean_data):
                action_dict[cle] = (cout, gain)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")            


In [9]:
action_dict
                

{'Action-1': (2000, 100),
 'Action-2': (3000, 300),
 'Action-3': (5000, 750),
 'Action-4': (7000, 1400),
 'Action-5': (6000, 1019),
 'Action-6': (8000, 2000),
 'Action-7': (2200, 154),
 'Action-8': (2600, 286),
 'Action-9': (4800, 624),
 'Action-10': (3400, 918),
 'Action-11': (4200, 714),
 'Action-12': (11000, 990),
 'Action-13': (3800, 874),
 'Action-14': (1400, 14),
 'Action-15': (1800, 54),
 'Action-16': (800, 64),
 'Action-17': (400, 48),
 'Action-18': (1000, 140),
 'Action-19': (2400, 504),
 'Action-20': (11400, 2052)}

Résolution en force brute récurrente:
si on considère l'ensemble des 'actions' du porte-feuille potentiel, il existe de multiples combinaisons d''action'. Parmi toutes les combinaisons dont le cout est inférieur au budget d'investissement, l'une d'entre elles est optimale car elle fournit le plus grand profit.


Maintenant quand on prend une 'action' au hasard,
soit on selectionne cette 'action' comme partie de la solution
soit on ne la selectionne pas.

Quand on selectionne une 'action', il faut exprimer la valeur et le cout du porte-feuille en fonction de sa valeur avant sélection (pour introduire de la récurrence avec n fonction de n-1).
valeur(pf(i)) = valeur(pf(i-1)) + valeur(action(i))
cout(pf(i)) = cout(pf(i-1)) + cout(action(i))
ou avec un cout exprimé en budget restant:
budget_restant(pf(i)) = budget_restant(pf(i-1)) - cout(action(i))

Quand on ne selectionne pas une action, la valeur et le poids du porte-feuille sont inchangés.

Cas d'arrêt de la fonction récurrente : si tout le budget est épuisé ou si toutes les actions ont été considérées.

Formalisons un peu mieux:

selectionne_pour_portefeuille(liste_action, budget)


In [39]:
# This code is contributed by Nikhil Kumar Singh
def knapSack(W, wt, val, n, item_list):
 
    # Base Case
    if n == 0 or W == 0:
        return 0, item_list
 
    # If weight of the nth item is
    # more than Knapsack of capacity W,
    # then this item cannot be included
    # in the optimal solution
    if (wt[n-1] > W):
        return knapSack(W, wt, val, n-1, item_list)
 
    # return the maximum of two cases:
    # (1) nth item included
    # (2) not included
    else:
        # take the item
        included = val[n-1] + knapSack(
                W-wt[n-1], wt, val, n-1, item_list)[0]
        excluded = knapSack(W, wt, val, n-1, item_list)[0]
        if included > excluded:
            if n not in item_list:
                item_list.append(n)
            optimum = included
        else:
            optimum = excluded
            if n in item_list:
                item_list.remove(n)
        return (optimum, item_list)
 
# end of function knapSack

In [41]:
val = [profit[1] for _,profit in action_dict.items()]
wt =  [profit[0] for _,profit in action_dict.items()]
W = BUDGET
n = len(val)
valeur, action_pf = knapSack(W, wt, val, n, [])
print(action_pf)

[1, 2, 3, 4, 6, 13, 18, 10, 19, 11, 5, 9, 8, 7, 12, 20]


une approche récursive avec mémorisation des sous-arbres déja vu une fois

# This code is contributed by Prosun Kumar Sarkar

In [51]:

 
 
def knapsack_memorize(wt, val, W, n):
 
    # base conditions
    if n == 0 or W == 0:
        return 0
    # already computed earlier
    if t[n][W] != -1:
        return t[n][W]
 
    # choice diagram code
    if wt[n-1] <= W:
        t[n][W] = max(
            val[n-1] + knapsack_memorize(
                wt, val, W-wt[n-1], n-1),
            knapsack_memorize(wt, val, W, n-1))
        return t[n][W]
    elif wt[n-1] > W:
        t[n][W] = knapsack_memorize(wt, val, W, n-1)
        return t[n][W]

In [48]:
# We initialize the matrix with -1 at first.
t = [[-1 for i in range(W + 1)] for j in range(n + 1)]

In [54]:
t[0][0]

-1

In [57]:
val = [profit[1] for _,profit in action_dict.items()]
wt =  [profit[0] for _,profit in action_dict.items()]

n = len(val)
knapsack_memorize(wt, val,BUDGET, n)


9907

### 2. Calcul du tableau

n (=956 ou =20) lignes x budget (=500 +1) colonnes
environ 501 000 points mémoire pour l'exemple complet n°2. 
et 10 000 points dans l'exemple n°1.


exemple de résolution:
[algorithm => Problème de sac à dos](https://learntutorials.net/fr/algorithm/topic/7250/probleme-de-sac-a-dos)

Time Complexité du code ci-dessus: `O(nW)` où n est le nombre d'éléments et W la capacité du sac à dos.

   Valeurs (tableau v)
   Poids (tableau w)
   Nombre d'éléments distincts (n)
   Capacité (W) 

def knapSack(W, wt, val, n):
    K = [[0 for x in range(W+1)] for x in range(n+1)]
    for i in range(n+1):
        for w in range(W+1):
            if i==0 or w==0:
                K[i][w] = 0
            elif wt[i-1] <= w:
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
    return K[n][W]
val = [60, 100, 120]
wt = [10, 20, 30]
W = 50
n = len(val)
print(knapSack(W, wt, val, n))



In [19]:
superK = [[0 for x in range(W+1)] for x in range(n+1)]
def knapSack(W, wt, val, n):
    K = [[0 for x in range(W+1)] for x in range(n+1)]
    for i in range(n+1):
        for w in range(W+1):
            if i==0 or w==0:
                K[i][w] = 0
            elif wt[i-1] <= w:
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w])
              
            else:
                K[i][w] = K[i-1][w]
    superK = K        
    return K[n][W]


In [20]:
val = [profit[1] for _,profit in action_dict.items()]
wt =  [profit[0] for _,profit in action_dict.items()]
W = BUDGET
n = len(val)
print(knapSack(W, wt, val, n))

9907


In [14]:
# remplir le tableau
KSP = [[ligne,0] for ligne in range(int(BUDGET/100))]

